### 상대모멘텀 
- 10개 주식 데이터를 이용
1. 월초부터 월말의 수정주가를 이용하여 월별 수익율
2. 월말의 수익율이 높은 순서 중 1개의 주식을 선택
3. 자른 종목만 투자하는 경우 수익율 계산

In [ ]:
import pandas as pd
from datetime import datetime

In [ ]:
# 월별 수익율 
def create_1M_rtn(sample, ticker, start = '2010-01-01', col = 'Adj Close'):
    # CODE라는 컬럼을 생성 -> ticker 대입 
    sample['CODE'] = ticker
    if 'Date' in sample.columns:
        # copy() : 복사본 생성 
        sample = sample.loc[sample['Date'] >= start, 
                            ['Date', 'CODE', col]].copy()
        # sample의 Date를 시계열로 변경하고 인덱스로 변경
        sample['Date'] = pd.to_datetime(sample['Date'])
        sample.set_index('Date', inplace=True)
    else:
        sample.index = pd.to_datetime(sample.index)
        sample = sample.loc[start:, ['CODE', col]].copy()
    # 기준년월 컬럼을 생성 
    sample['STD-YM'] = sample.index.map(
        lambda x : x.strftime('%Y-%m')
    )
    sample['1m_rtn'] = 0
    #  중복데이터를 제거하고 고유한 값들만 list형태로 생성
    ym_list = sample['STD-YM'].unique() 
    return sample, ym_list


In [ ]:
df = pd.read_csv('../../csv/AAPL.csv')
df2 = pd.read_csv('../../csv/AMZN.csv', index_col='Date')

In [ ]:
sample_df, ym_list = create_1M_rtn(df, 'AAPL')

In [ ]:
sample_df.head()

In [ ]:
ym_list

In [ ]:
sample_df2, ym_list2 = create_1M_rtn(df2, 'AMZM')

In [ ]:
sample_df2.head()

In [ ]:
ym_list2

In [ ]:
def create_trade_book(sample, code):
    # 새로운 데이터프레임을 생성
    book = pd.DataFrame()
    book = sample[code].copy()
    book['STD-YM'] = book.index.map(
        lambda x : x.strftime('%Y-%m')
    )
    for c in code:
        book['p'+c] = ''
        book['r'+c] = ''
    return book

In [ ]:
import os 
import glob
import numpy as np

In [ ]:
# 특정 경로에 있는 파일의 목록을 불러오기 
os.listdir("../../data/")

files = glob.glob("../../data/*.csv")

# 새로운 데이터프레임을 생성 
# 종목별 데이터프레임
stock_df = pd.DataFrame()
# 월말 데이터프레임 
month_last_df = pd.DataFrame()

for file in files:
    folder, name = os.path.split(file)
    # print(folder, name)
    head, tail = os.path.splitext(name)
    # print(head, tail)
    # head를 create_1M_rtn 함수에 ticker로 이용하기 위해 생성

    # 파일 로드 
    read_df = pd.read_csv(file)

    # 첫번째 함수를 로드 
    price_df, ym_list = create_1M_rtn(read_df, head)

    # 데이터프레임을 모두 결합
    stock_df = pd.concat([stock_df, price_df], axis=0)

    # 월별 상대모멘텀을 계산하기 위해 1개월간의 수익율 계산
    for ym in ym_list:
        flag = price_df['STD-YM'] == ym
        m_rtn = price_df.loc[flag].iloc[-1, 1] / price_df.loc[flag].iloc[0, 1]
        price_df.loc[flag, '1m_rtn'] = m_rtn
        data = price_df.loc[flag, ['CODE', '1m_rtn']].tail(1)
        month_last_df = pd.concat([month_last_df, data], axis=0)

In [ ]:
month_last_df

In [ ]:
# warning 메시지 제거 
import warnings
warnings.filterwarnings('ignore')

In [ ]:
test = stock_df.copy()

In [ ]:
# pivot_table() : groupby()와 흡사
test = test[['CODE', 'Adj Close', '1m_rtn']]

In [ ]:
test.reset_index(inplace=True)

In [ ]:
test.groupby(['Date','CODE']).mean().unstack()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
test.pivot_table(
    index='Date', 
    columns= 'CODE', 
    values= 'Adj Close', 
    aggfunc= ['mean', 'sum']
)

In [ ]:
month_rtn_df = month_last_df.copy()

In [ ]:
month_rtn_df.reset_index(inplace=True)

In [ ]:
month_rtn_df = month_rtn_df[['Date', 'CODE', '1m_rtn']]

In [ ]:
month_rtn_df = month_rtn_df.pivot_table(index='Date', columns='CODE', values='1m_rtn')

In [ ]:
month_rtn_df.head(1)

In [ ]:
month_rtn_df = month_rtn_df.rank(axis=1, ascending=False, method='max', pct=True)

In [ ]:
# 상위 15% 종목들만 선택
month_rtn_df = month_rtn_df.where(month_rtn_df < 0.15, 0)

In [ ]:
# 데이터 중 0 이 아니면 1로 대체
month_rtn_df[month_rtn_df != 0] = 1

In [ ]:
month_rtn_df

In [ ]:
stock_codes = stock_df['CODE'].unique()

In [ ]:
stock_codes

In [ ]:
month_rtn_df.loc['2010-01-29 00:00:00', month_rtn_df.iloc[0] >= 1].index

In [ ]:
sig_dict = dict()

for date in month_rtn_df.index:
    # 데이터가 1인 종목코드를 로드 
    ticker_list = list(month_rtn_df.loc[date, month_rtn_df.loc[date] >= 1].index)
    # print(ticker_list)
    sig_dict[date] = ticker_list
sig_dict

In [ ]:
stock_df.head()

In [ ]:
stock_c_matrix = stock_df.reset_index().pivot_table(
    index='Date', 
    columns='CODE', 
    values=stock_df.columns[1])
stock_c_matrix

In [ ]:
book = create_trade_book(stock_c_matrix, stock_codes)


In [136]:
book.columns

Index(['AAPL', 'AMZN', 'BND', 'GDX', 'GLD', 'GM', 'MSFT', 'SLV', 'SPY', 'USM',
       'STD-YM', 'pAAPL', 'rAAPL', 'pAMZN', 'rAMZN', 'pBND', 'rBND', 'pGDX',
       'rGDX', 'pGLD', 'rGLD', 'pGM', 'rGM', 'pMSFT', 'rMSFT', 'pSLV', 'rSLV',
       'pSPY', 'rSPY', 'pUSM', 'rUSM'],
      dtype='object', name='CODE')

In [137]:
# 세번째 함수 
def trading(book, s_code):
    std_ym = ''
    buy_phase = False

    # 종목별 순회 
    for code in s_code:
        for i in book.index:
            # 해당 종목코드 포지션을 잡아준다.
            if (book.loc[i, 'p'+code] == '') & \
                  (book.shift(1).loc[i, 'p'+code] == 'ready'+code):
                std_ym = book.loc[i, 'STD-YM']
                buy_phase = True
            # 해당 종목코드에서 신호가 잡혀있다면 매수 상태를 유지 
            if (book.loc[i, 'p'+code] == '') & \
                (book.loc[i, 'STD-YM'] == std_ym) & \
                (buy_phase):
                book.loc[i, 'p'+code] = 'buy'+code

            # std_ym, buy_phase 초기화
            if book.loc[i, 'p'+code] == '':
                std_ym = None
                buy_phase = False
    return book


In [138]:
sig_dict.items()

dict_items([(Timestamp('2010-01-29 00:00:00'), ['BND']), (Timestamp('2010-02-26 00:00:00'), ['AAPL']), (Timestamp('2010-03-31 00:00:00'), ['AAPL']), (Timestamp('2010-04-30 00:00:00'), ['AAPL']), (Timestamp('2010-05-28 00:00:00'), ['GLD']), (Timestamp('2010-06-30 00:00:00'), ['GDX']), (Timestamp('2010-07-30 00:00:00'), ['USM']), (Timestamp('2010-08-31 00:00:00'), ['GDX']), (Timestamp('2010-09-30 00:00:00'), ['AMZN']), (Timestamp('2010-10-29 00:00:00'), ['SLV']), (Timestamp('2010-11-30 00:00:00'), ['SLV']), (Timestamp('2010-12-31 00:00:00'), ['SLV']), (Timestamp('2011-01-31 00:00:00'), ['AAPL']), (Timestamp('2011-02-28 00:00:00'), ['SLV']), (Timestamp('2011-03-31 00:00:00'), ['SLV']), (Timestamp('2011-04-29 00:00:00'), ['SLV']), (Timestamp('2011-05-31 00:00:00'), ['BND']), (Timestamp('2011-06-30 00:00:00'), ['MSFT']), (Timestamp('2011-07-29 00:00:00'), ['SLV']), (Timestamp('2011-08-31 00:00:00'), ['GLD']), (Timestamp('2011-09-30 00:00:00'), ['AMZN']), (Timestamp('2011-10-31 00:00:00'), [

In [139]:
# 포지션을 잡는 부분 
for date, values in sig_dict.items():
    for stock in values:
        book.loc[date, 'p'+stock] = 'ready'+stock

In [140]:
book = trading(book, stock_codes)

book.head()

UnboundLocalError: cannot access local variable 'std_ym' where it is not associated with a value